# Feature Store SDK Demo

This notebook demonstrates the complete functionality of our custom Feature Store SDK.

## Features:
- ✅ Delta Lake storage format
- ✅ Automatic joins between feature groups
- ✅ Precise feature selection via projections
- ✅ **Clean filter syntax: Tuple `("age", ">", 30)` format**
- ✅ Multiple output formats: Spark, Pandas, Polars
- ✅ Simple API without over-engineering

## Setup and Imports

In [ ]:
import os
import sys
import pandas as pd
import polars as pl
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip

# Add the parent directory to Python path to import our SDK
sys.path.append('/workspace')
from feature_store_sdk import FeatureStore, projection

print("📦 All imports successful!")

## Initialize Spark with Delta Lake

In [ ]:
# Initialize Spark with Delta Lake support
builder = SparkSession.builder.appName("FeatureStoreSDKDemo") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()
spark.sparkContext.setLogLevel("WARN")

print(f"✅ Spark {spark.version} initialized with Delta Lake support")
print(f"🌐 Spark UI: http://localhost:4040")

## Create Sample Business Data

Let's create realistic business data for our feature store demo.

In [ ]:
# Create sample business data
print("📊 Creating sample business data...")

# Customer accounts data
accounts_data = pd.DataFrame({
    'account_id': ['ACC001', 'ACC002', 'ACC003', 'ACC004', 'ACC005', 'ACC006'],
    'user_id': ['USER001', 'USER002', 'USER003', 'USER004', 'USER005', 'USER006'],
    'account_type': ['PREMIUM', 'STANDARD', 'PREMIUM', 'GOLD', 'STANDARD', 'GOLD'],
    'status': ['ACTIVE', 'ACTIVE', 'INACTIVE', 'ACTIVE', 'ACTIVE', 'SUSPENDED'],
    'opened_at': ['2023-01-15', '2023-02-20', '2023-03-10', '2023-04-05', '2023-05-12', '2023-06-01'],
    'credit_limit': [10000, 5000, 15000, 25000, 3000, 20000]
})

# User profile data
users_data = pd.DataFrame({
    'user_id': ['USER001', 'USER002', 'USER003', 'USER004', 'USER005', 'USER006'],
    'age': [25, 34, 28, 45, 33, 39],
    'segment': ['PREMIUM', 'STANDARD', 'PREMIUM', 'GOLD', 'STANDARD', 'GOLD'],
    'country': ['US', 'UK', 'CA', 'US', 'DE', 'FR'],
    'city': ['New York', 'London', 'Toronto', 'San Francisco', 'Berlin', 'Paris'],
    'income_bracket': ['HIGH', 'MEDIUM', 'HIGH', 'VERY_HIGH', 'MEDIUM', 'HIGH'],
    'signup_date': ['2022-12-01', '2023-01-15', '2023-02-01', '2022-11-15', '2023-04-01', '2023-05-20']
})

# Transaction profile data (aggregated features)
transactions_data = pd.DataFrame({
    'account_id': ['ACC001', 'ACC002', 'ACC003', 'ACC004', 'ACC005', 'ACC006'],
    'last_txn_ts': ['2024-01-15 10:30:00', '2024-01-14 15:45:00', '2023-12-20 09:15:00', 
                   '2024-01-16 14:20:00', '2024-01-15 11:55:00', '2024-01-13 16:30:00'],
    'avg_ticket': [125.50, 89.75, 245.30, 67.80, 156.25, 301.40],
    'txn_cnt_30d': [8, 5, 1, 12, 7, 15],
    'txn_cnt_90d': [15, 8, 2, 22, 12, 28],
    'total_spend_90d': [1882.5, 718.0, 490.6, 1491.6, 1875.0, 8439.2],
    'distinct_merchants_90d': [8, 5, 2, 12, 7, 16]
})

# Risk scores (additional feature group)
risk_data = pd.DataFrame({
    'account_id': ['ACC001', 'ACC002', 'ACC003', 'ACC004', 'ACC005', 'ACC006'],
    'credit_score': [750, 680, 720, 800, 650, 780],
    'fraud_score': [0.05, 0.12, 0.03, 0.01, 0.08, 0.02],
    'risk_category': ['LOW', 'MEDIUM', 'LOW', 'VERY_LOW', 'MEDIUM', 'LOW'],
    'last_risk_assessment': ['2024-01-10', '2024-01-12', '2023-12-15', '2024-01-14', '2024-01-11', '2024-01-09']
})

print(f"📋 Created {len(accounts_data)} accounts")
print(f"👥 Created {len(users_data)} user profiles") 
print(f"💳 Created {len(transactions_data)} transaction profiles")
print(f"⚠️ Created {len(risk_data)} risk assessments")

# Display sample data
print("\n📊 Sample accounts data:")
print(accounts_data.head(3))
print("\n👥 Sample users data:")
print(users_data.head(3))

## Save Data as Delta Tables

In [ ]:
# Save all data as Delta Lake tables
base_path = "/workspace/data/feature_store_demo"
print(f"💾 Saving data to Delta Lake at: {base_path}")

# Convert to Spark DataFrames and save
accounts_df = spark.createDataFrame(accounts_data)
accounts_df.write.format("delta").mode("overwrite").save(f"{base_path}/accounts")
print("✅ Accounts saved")

users_df = spark.createDataFrame(users_data)  
users_df.write.format("delta").mode("overwrite").save(f"{base_path}/users")
print("✅ Users saved")

transactions_df = spark.createDataFrame(transactions_data)
transactions_df.write.format("delta").mode("overwrite").save(f"{base_path}/transactions_profile")
print("✅ Transaction profiles saved")

risk_df = spark.createDataFrame(risk_data)
risk_df.write.format("delta").mode("overwrite").save(f"{base_path}/risk_scores")
print("✅ Risk scores saved")

print("\n🎉 All data successfully saved in Delta Lake format!")

## Initialize Feature Store SDK

Now let's use our SDK to create feature groups and feature views.

In [ ]:
# Initialize Feature Store
fs = FeatureStore(spark=spark)
print("✅ Feature Store initialized")

# Create feature groups with explicit data locations
print("\n📊 Creating feature groups...")

accounts_fg = fs.get_or_create_batch_feature_group(
    name="accounts", 
    version=1, 
    keys=["account_id"],
    data_location=f"{base_path}/accounts",
    description="Customer account information"
)
print(f"✅ {accounts_fg}")

users_fg = fs.get_or_create_batch_feature_group(
    name="users", 
    version=1, 
    keys=["user_id"],
    data_location=f"{base_path}/users",
    description="User demographic and profile data"
)
print(f"✅ {users_fg}")

transactions_fg = fs.get_or_create_batch_feature_group(
    name="transactions_profile", 
    version=1, 
    keys=["account_id"],
    data_location=f"{base_path}/transactions_profile",
    description="Aggregated transaction features per account"
)
print(f"✅ {transactions_fg}")

risk_fg = fs.get_or_create_batch_feature_group(
    name="risk_scores", 
    version=1, 
    keys=["account_id"],
    data_location=f"{base_path}/risk_scores",
    description="Risk assessment scores and categories"
)
print(f"✅ {risk_fg}")

print("\n🎯 All feature groups created successfully!")

## Test 1: Basic Feature Selection

Test that we can select specific features from individual feature groups.

In [ ]:
print("🧪 Test 1: Basic Feature Selection")
print("=" * 40)

# Create a simple feature view with only specific features
basic_fv = fs.get_or_create_feature_view(
    name="basic_account_features", 
    version=1, 
    base=accounts_fg,
    source_projections=[
        projection(
            source=accounts_fg,
            features=["account_id", "status", "account_type"]  # Only these 3 features
        )
    ],
    description="Basic account features - minimal set"
)

# Test the query
result = basic_fv.plan().to_pandas()
print(f"📋 Columns returned: {list(result.columns)}")
print(f"📊 Expected: ['account_id', 'status', 'account_type']")
print(f"✅ Feature selection working: {set(result.columns) == {'account_id', 'status', 'account_type'}}")
print(f"📈 Row count: {len(result)}")

print("\n📊 Sample data:")
print(result.head())

## Test 2: Multi-Table Join with Feature Selection

Test automatic joins between multiple feature groups with precise feature selection.

In [ ]:
print("🧪 Test 2: Multi-Table Join with Feature Selection")
print("=" * 50)

# Create comprehensive feature view with joins
comprehensive_fv = fs.get_or_create_feature_view(
    name="comprehensive_features", 
    version=1, 
    base=accounts_fg,
    source_projections=[
        # Base account features
        projection(
            source=accounts_fg,
            features=["account_id", "user_id", "status", "account_type", "credit_limit"]
        ),
        # User demographics - join on user_id
        projection(
            source=users_fg,
            features=["age", "segment", "country", "income_bracket"],
            keys_map={"user_id": "user_id"},
            join_type="left"
        ),
        # Transaction features - join on account_id
        projection(
            source=transactions_fg,
            features=["avg_ticket", "txn_cnt_90d", "total_spend_90d"],
            keys_map={"account_id": "account_id"},
            join_type="left"
        ),
        # Risk scores - join on account_id
        projection(
            source=risk_fg,
            features=["credit_score", "fraud_score", "risk_category"],
            keys_map={"account_id": "account_id"},
            join_type="left"
        )
    ],
    description="Comprehensive account features with user, transaction, and risk data"
)

# Test the comprehensive query
result = comprehensive_fv.plan().to_pandas()
print(f"📋 Columns returned: {list(result.columns)}")
print(f"📊 Total features: {len(result.columns)}")
print(f"📈 Row count: {len(result)}")

expected_cols = {
    'account_id', 'user_id', 'status', 'account_type', 'credit_limit',  # accounts
    'age', 'segment', 'country', 'income_bracket',  # users
    'avg_ticket', 'txn_cnt_90d', 'total_spend_90d',  # transactions
    'credit_score', 'fraud_score', 'risk_category'   # risk
}
print(f"✅ All expected features present: {set(result.columns) == expected_cols}")

print("\n📊 Sample comprehensive data:")
print(result.head(3))

## Test 3: Multiple Output Formats

Demonstrate that the same feature view can output to Spark, Pandas, and Polars.

In [ ]:
print("🧪 Test 3: Multiple Output Formats")
print("=" * 35)

# Create a focused feature view for format testing
format_test_fv = fs.get_or_create_feature_view(
    name="format_test_features", 
    version=1, 
    base=accounts_fg,
    source_projections=[
        projection(
            source=accounts_fg,
            features=["account_id", "status", "credit_limit"]
        ),
        projection(
            source=users_fg,
            features=["age", "country"],
            keys_map={"user_id": "user_id"},
            join_type="left"
        )
    ]
)

query_plan = format_test_fv.plan()

print("\n🔥 Testing Spark DataFrame output:")
spark_df = query_plan.to_spark(spark)
print(f"   Type: {type(spark_df)}")
print(f"   Columns: {spark_df.columns}")
print(f"   Count: {spark_df.count()}")
spark_df.show(3)

print("\n🐼 Testing Pandas DataFrame output:")
pandas_df = query_plan.to_pandas()
print(f"   Type: {type(pandas_df)}")
print(f"   Shape: {pandas_df.shape}")
print(f"   Columns: {list(pandas_df.columns)}")
print(pandas_df.head(3))

print("\n⚡ Testing Polars DataFrame output:")
polars_df = query_plan.to_polars()
print(f"   Type: {type(polars_df)}")
print(f"   Shape: {polars_df.shape}")
print(f"   Columns: {list(polars_df.columns)}")
print(polars_df.head(3))

print("\n✅ All output formats working correctly!")

## Test 4: Advanced Feature Engineering Scenario

Simulate a real-world ML scenario where we need specific features for model training.

In [ ]:
print("🧪 Test 4: Advanced Feature Engineering Scenario")
print("=" * 45)

# Scenario: Create features for a credit risk model
credit_risk_fv = fs.get_or_create_feature_view(
    name="credit_risk_model_features", 
    version=1, 
    base=accounts_fg,
    source_projections=[
        # Account basics
        projection(
            source=accounts_fg,
            features=["account_id", "account_type", "credit_limit", "status"]
        ),
        # Customer demographics for risk assessment
        projection(
            source=users_fg,
            features=["age", "income_bracket", "country"],
            keys_map={"user_id": "user_id"},
            join_type="left"
        ),
        # Transaction behavior patterns
        projection(
            source=transactions_fg,
            features=["txn_cnt_30d", "txn_cnt_90d", "avg_ticket", "total_spend_90d", "distinct_merchants_90d"],
            keys_map={"account_id": "account_id"},
            join_type="left"
        ),
        # Risk indicators
        projection(
            source=risk_fg,
            features=["credit_score", "fraud_score", "risk_category"],
            keys_map={"account_id": "account_id"},
            join_type="left"
        )
    ],
    description="Features for credit risk modeling"
)

# Get features as Polars for fast processing
ml_features = credit_risk_fv.plan().to_polars()

print(f"📊 ML Feature Set created:")
print(f"   Features: {len(ml_features.columns)}")
print(f"   Samples: {len(ml_features)}")
print(f"   Feature names: {list(ml_features.columns)}")

print("\n📈 Feature Statistics:")
print(ml_features.describe())

print("\n🎯 Ready for ML model training!")
print("\n📋 Sample ML training data:")
print(ml_features.head())

## Test 5: Performance and Query Plan Analysis

Examine the underlying Spark execution plan and performance characteristics.

In [ ]:
print("🧪 Test 5: Performance and Query Plan Analysis")
print("=" * 45)

# Get the Spark DataFrame to analyze execution plan
spark_result = comprehensive_fv.plan().to_spark(spark)

print("🔍 Spark Execution Plan:")
print("=" * 25)
spark_result.explain(True)

print("\n📊 Query Performance Metrics:")
print(f"   Total columns: {len(spark_result.columns)}")
print(f"   Total rows: {spark_result.count()}")

print("\n🏗️ Data Sources Verified:")
print(f"   ✅ Accounts FG exists: {accounts_fg.exists()}")
print(f"   ✅ Users FG exists: {users_fg.exists()}")
print(f"   ✅ Transactions FG exists: {transactions_fg.exists()}")
print(f"   ✅ Risk FG exists: {risk_fg.exists()}")

print("\n📋 Schema Information:")
spark_result.printSchema()

## Test 6: Filter Functionality

Test the new filter functionality in source_projections.

In [ ]:
print("🧪 Test 6: Filter Functionality")
print("=" * 32)

# Test 6.1: Single equality filter - only ACTIVE accounts (tuple format)
print("\n📋 Test 6.1: Single Equality Filter - Tuple Format")
active_accounts_fv = fs.get_or_create_feature_view(
    name="active_accounts_only", 
    version=1, 
    base=accounts_fg,
    source_projections=[
        projection(
            source=accounts_fg,
            features=["account_id", "status", "account_type", "credit_limit"],
            filters=("status", "==", "ACTIVE")
        )
    ],
    description="Only active accounts"
)

active_result = active_accounts_fv.plan().to_pandas()
print(f"📊 Original accounts: {len(accounts_data)}")
print(f"📊 Active accounts only: {len(active_result)}")
print(f"✅ All accounts are ACTIVE: {all(active_result['status'] == 'ACTIVE')}")
print(active_result)

# Test Spark output for active accounts
print("\n🔥 Testing Spark output for filtered data:")
active_spark = active_accounts_fv.plan().to_spark(spark)
print(f"   Spark DataFrame columns: {active_spark.columns}")
print(f"   Spark DataFrame count: {active_spark.count()}")
active_spark.show(3)

# Test Polars output for active accounts
print("\n⚡ Testing Polars output for filtered data:")
active_polars = active_accounts_fv.plan().to_polars()
print(f"   Polars DataFrame type: {type(active_polars)}")
print(f"   Polars DataFrame shape: {active_polars.shape}")
print(f"   Polars DataFrame columns: {list(active_polars.columns)}")
print(f"   ✅ Polars filter working: {all(active_polars['status'] == 'ACTIVE')}")
print("   Sample Polars data:")
print(active_polars.head(3))

# Test 6.2: Range filter - age > 30 (tuple format)
print("\n📋 Test 6.2: Range Filter - Tuple Format")
mature_users_fv = fs.get_or_create_feature_view(
    name="mature_users_features", 
    version=1, 
    base=accounts_fg,
    source_projections=[
        # Base accounts
        projection(
            source=accounts_fg,
            features=["account_id", "user_id", "account_type"]
        ),
        # Users with age filter using tuple format
        projection(
            source=users_fg,
            features=["age", "country", "income_bracket"],
            keys_map={"user_id": "user_id"},
            join_type="left",
            filters=("age", ">", 30)  # Tuple format: (column, operator, value)
        )
    ],
    description="Accounts with users over 30"
)

mature_result = mature_users_fv.plan().to_pandas()
mature_ages = mature_result['age'].dropna()
print(f"📊 Users with age > 30: {len(mature_ages)}")
print(f"✅ All ages > 30: {all(mature_ages > 30)}")
print(f"📈 Age range: {mature_ages.min():.0f} - {mature_ages.max():.0f}")
print(mature_result.head())

# Test Spark output for age filter
print("\n🔥 Testing Spark output for age filter:")
mature_spark = mature_users_fv.plan().to_spark(spark)
print(f"   Spark DataFrame columns: {mature_spark.columns}")
print(f"   Spark DataFrame count: {mature_spark.count()}")
mature_spark.show(3)

# Test Polars output for age filter
print("\n⚡ Testing Polars output for age filter:")
mature_polars = mature_users_fv.plan().to_polars()
mature_polars_ages = mature_polars.filter(mature_polars['age'].is_not_null())['age']
print(f"   Polars DataFrame type: {type(mature_polars)}")
print(f"   Polars DataFrame shape: {mature_polars.shape}")
print(f"   Polars DataFrame columns: {list(mature_polars.columns)}")
print(f"   ✅ Polars age filter working: {all(mature_polars_ages > 30) if len(mature_polars_ages) > 0 else True}")
print("   Sample Polars data:")
print(mature_polars.head(3))

# Test 6.3: IN filter - specific countries (tuple format)
print("\n📋 Test 6.3: IN Filter - Tuple Format")
us_uk_fv = fs.get_or_create_feature_view(
    name="us_uk_accounts", 
    version=1, 
    base=accounts_fg,
    source_projections=[
        # Base accounts
        projection(
            source=accounts_fg,
            features=["account_id", "user_id", "status"]
        ),
        # Users from US or UK only using tuple format
        projection(
            source=users_fg,
            features=["country", "age", "segment"],
            keys_map={"user_id": "user_id"},
            join_type="left",
            filters=("country", "in", ["US", "UK"])  # Tuple format for IN filter
        )
    ],
    description="Accounts from US and UK users"
)

us_uk_result = us_uk_fv.plan().to_pandas()
countries = us_uk_result['country'].dropna().unique()
print(f"📊 Countries found: {list(countries)}")
print(f"✅ Only US/UK: {set(countries).issubset({'US', 'UK'})}")
print(us_uk_result)

# Test Spark output for IN filter
print("\n🔥 Testing Spark output for IN filter:")
us_uk_spark = us_uk_fv.plan().to_spark(spark)
print(f"   Spark DataFrame columns: {us_uk_spark.columns}")
print(f"   Spark DataFrame count: {us_uk_spark.count()}")
us_uk_spark.show(3)

# Test Polars output for IN filter
print("\n⚡ Testing Polars output for IN filter:")
us_uk_polars = us_uk_fv.plan().to_polars()
polars_countries = us_uk_polars.filter(us_uk_polars['country'].is_not_null())['country'].unique().to_list()
print(f"   Polars DataFrame type: {type(us_uk_polars)}")
print(f"   Polars DataFrame shape: {us_uk_polars.shape}")
print(f"   Polars DataFrame columns: {list(us_uk_polars.columns)}")
print(f"   ✅ Polars IN filter working: {set(polars_countries).issubset({'US', 'UK'}) if len(polars_countries) > 0 else True}")
print("   Sample Polars data:")
print(us_uk_polars.head(3))

# Test 6.4: Multiple filters using tuple format
print("\n📋 Test 6.4: Multiple Filters - Clean Tuple Format")
low_risk_high_credit_fv = fs.get_or_create_feature_view(
    name="low_risk_high_credit", 
    version=1, 
    base=accounts_fg,
    source_projections=[
        projection(
            source=accounts_fg,
            features=["account_id", "status", "credit_limit"]
        ),
        projection(
            source=risk_fg,
            features=["credit_score", "risk_category", "fraud_score"],
            keys_map={"account_id": "account_id"},
            join_type="left",
            filters=[  # Multiple filters using tuple format - much cleaner!
                ("credit_score", ">", 700),
                ("risk_category", "==", "LOW")
            ]
        )
    ],
    description="High credit score, low risk accounts"
)

filtered_result = low_risk_high_credit_fv.plan().to_pandas()
credit_scores = filtered_result['credit_score'].dropna()
risk_cats = filtered_result['risk_category'].dropna()

print(f"📊 Accounts matching criteria: {len(filtered_result)}")
print(f"✅ All credit scores > 700: {all(credit_scores > 700) if len(credit_scores) > 0 else 'No data'}")
print(f"✅ All risk categories LOW: {all(risk_cats == 'LOW') if len(risk_cats) > 0 else 'No data'}")
print(filtered_result)

# Test Spark output for multiple filters
print("\n🔥 Testing Spark output for multiple filters:")
filtered_spark = low_risk_high_credit_fv.plan().to_spark(spark)
print(f"   Spark DataFrame columns: {filtered_spark.columns}")
print(f"   Spark DataFrame count: {filtered_spark.count()}")
filtered_spark.show(3)

# Test Polars output for multiple filters
print("\n⚡ Testing Polars output for multiple filters:")
filtered_polars = low_risk_high_credit_fv.plan().to_polars()
polars_credit_scores = filtered_polars.filter(filtered_polars['credit_score'].is_not_null())['credit_score']
polars_risk_cats = filtered_polars.filter(filtered_polars['risk_category'].is_not_null())['risk_category']
print(f"   Polars DataFrame type: {type(filtered_polars)}")
print(f"   Polars DataFrame shape: {filtered_polars.shape}")
print(f"   Polars DataFrame columns: {list(filtered_polars.columns)}")
print(f"   ✅ Polars multiple filters working: Credit scores > 700: {all(polars_credit_scores > 700) if len(polars_credit_scores) > 0 else True}")
print(f"   ✅ Polars multiple filters working: Risk categories LOW: {all(polars_risk_cats == 'LOW') if len(polars_risk_cats) > 0 else True}")
print("   Sample Polars data:")
print(filtered_polars.head(3))

# Test 6.5: Complex scenario using tuple format
print("\n📋 Test 6.5: Complex Business Scenario - Pure Tuple Format")
premium_high_spenders_fv = fs.get_or_create_feature_view(
    name="premium_high_spenders", 
    version=1, 
    base=accounts_fg,
    source_projections=[
        # Tuple format for base table
        projection(
            source=accounts_fg,
            features=["account_id", "user_id", "account_type", "credit_limit"],
            filters=("account_type", "==", "PREMIUM")
        ),
        # Tuple format for transaction data
        projection(
            source=transactions_fg,
            features=["total_spend_90d", "txn_cnt_90d", "avg_ticket"],
            keys_map={"account_id": "account_id"},
            join_type="left",
            filters=("total_spend_90d", ">", 1000)  # Clean tuple format
        ),
        # User demographics without filters
        projection(
            source=users_fg,
            features=["age", "income_bracket", "country"],
            keys_map={"user_id": "user_id"},
            join_type="left"
        )
    ],
    description="Premium accounts with high spending patterns"
)

business_result = premium_high_spenders_fv.plan().to_pandas()
spending = business_result['total_spend_90d'].dropna()
account_types = business_result['account_type'].dropna()

print(f"📊 Premium high-spender accounts: {len(business_result)}")
print(f"✅ All accounts are PREMIUM: {all(account_types == 'PREMIUM') if len(account_types) > 0 else 'No data'}")
print(f"✅ All spending > 1000: {all(spending > 1000) if len(spending) > 0 else 'No data'}")
print(f"💰 Average spending: ${spending.mean():.2f}" if len(spending) > 0 else "💰 No spending data")
print("\n📊 Premium High-Spender Profile:")
print(business_result)

# Test Spark output for complex scenario
print("\n🔥 Testing Spark output for complex business scenario:")
business_spark = premium_high_spenders_fv.plan().to_spark(spark)
print(f"   Spark DataFrame columns: {business_spark.columns}")
print(f"   Spark DataFrame count: {business_spark.count()}")
business_spark.show(3)

# Test Polars output for complex scenario
print("\n⚡ Testing Polars output for complex business scenario:")
business_polars = premium_high_spenders_fv.plan().to_polars()
polars_spending = business_polars.filter(business_polars['total_spend_90d'].is_not_null())['total_spend_90d']
polars_account_types = business_polars.filter(business_polars['account_type'].is_not_null())['account_type']
print(f"   Polars DataFrame type: {type(business_polars)}")
print(f"   Polars DataFrame shape: {business_polars.shape}")
print(f"   Polars DataFrame columns: {list(business_polars.columns)}")
print(f"   ✅ Polars complex filters working: All PREMIUM: {all(polars_account_types == 'PREMIUM') if len(polars_account_types) > 0 else True}")
print(f"   ✅ Polars complex filters working: All spending > 1000: {all(polars_spending > 1000) if len(polars_spending) > 0 else True}")
print("   Sample Polars data:")
print(business_polars.head(3))

# Test 6.6: Showcase all tuple format capabilities
print("\n📋 Test 6.6: Complete Tuple Format Showcase")
print("All filter types using the concise tuple syntax")

tuple_showcase_fv = fs.get_or_create_feature_view(
    name="tuple_format_showcase", 
    version=1, 
    base=accounts_fg,
    source_projections=[
        projection(
            source=accounts_fg,
            features=["account_id", "account_type"],
            filters=[  # Multiple tuple filters
                ("status", "==", "ACTIVE"),           # Equality
                ("credit_limit", ">=", 5000)         # Range
            ]
        ),
        projection(
            source=users_fg,
            features=["age", "country"],
            keys_map={"user_id": "user_id"},
            join_type="left",
            filters=[
                ("age", ">", 25),                    # Greater than
                ("country", "in", ["US", "UK", "CA"]) # IN filter
            ]
        )
    ],
    description="Demonstrating all tuple filter types"
)

tuple_result = tuple_showcase_fv.plan().to_pandas()
print(f"📊 Accounts with multiple tuple filters: {len(tuple_result)}")
print("✅ Tuple syntax examples:")
print('   - Equality: ("status", "==", "ACTIVE")')
print('   - Range: ("credit_limit", ">=", 5000)')
print('   - Greater than: ("age", ">", 25)')
print('   - IN filter: ("country", "in", ["US", "UK", "CA"])')
print(tuple_result)

# Test Spark output for complete showcase
print("\n🔥 Testing Spark output for tuple format showcase:")
tuple_spark = tuple_showcase_fv.plan().to_spark(spark)
print(f"   Spark DataFrame columns: {tuple_spark.columns}")
print(f"   Spark DataFrame count: {tuple_spark.count()}")
tuple_spark.show(3)

# Test Polars output for complete showcase
print("\n⚡ Testing Polars output for tuple format showcase:")
tuple_polars = tuple_showcase_fv.plan().to_polars()
print(f"   Polars DataFrame type: {type(tuple_polars)}")
print(f"   Polars DataFrame shape: {tuple_polars.shape}")
print(f"   Polars DataFrame columns: {list(tuple_polars.columns)}")
print("   Sample Polars data:")
print(tuple_polars.head(3))

print("\n🎯 Filter Functionality Tests Complete!")
print("✅ Tuple format: ('status', '==', 'ACTIVE')  # Clean and concise!")
print("✅ Multiple filters with tuples: [('age', '>', 30), ('country', 'in', ['US'])]")
print("✅ All operators work with tuple formats")
print("✅ Complex business scenarios with clean, readable filters")
print("✅ Spark DataFrame output works with all filter types")
print("✅ Polars DataFrame output works with all filter types (using lazy evaluation)")
print("✅ Pandas DataFrame output works with all filter types")

## SDK Validation Summary

Let's run a comprehensive validation of all SDK features including the new filter functionality.

In [ ]:
print("🏆 Feature Store SDK Validation Summary")
print("=" * 50)

# Test checklist
tests_passed = 0
total_tests = 0

def validate_test(condition, description):
    global tests_passed, total_tests
    total_tests += 1
    if condition:
        tests_passed += 1
        print(f"✅ {description}")
    else:
        print(f"❌ {description}")
    return condition

print("\n📋 Core Functionality Tests:")

# Test 1: FeatureStore initialization
validate_test(fs is not None, "FeatureStore initialization")

# Test 2: Feature group creation with data location
validate_test(accounts_fg.exists(), "Feature group creation and Delta Lake storage")

# Test 3: Basic feature selection
basic_result = basic_fv.plan().to_pandas()
validate_test(
    set(basic_result.columns) == {'account_id', 'status', 'account_type'},
    "Precise feature selection from projections"
)

# Test 4: Multi-table automatic joins
comp_result = comprehensive_fv.plan().to_pandas()
validate_test(
    len(comp_result.columns) == 15 and len(comp_result) == 6,
    "Multi-table automatic joins with feature selection"
)

# Test 5: Multiple output formats
try:
    test_plan = format_test_fv.plan()
    spark_out = test_plan.to_spark(spark)
    pandas_out = test_plan.to_pandas()
    polars_out = test_plan.to_polars()
    formats_work = all([
        len(spark_out.columns) > 0,
        len(pandas_out.columns) > 0,
        len(polars_out.columns) > 0
    ])
    validate_test(formats_work, "Multiple output formats (Spark/Pandas/Polars)")
except Exception as e:
    validate_test(False, f"Multiple output formats - Error: {e}")

# Test 6: Join key mapping
user_joined = any('age' in col for col in comp_result.columns)
validate_test(user_joined, "Custom join key mapping (account.user_id -> users.user_id)")

# Test 7: Different join types
validate_test(
    len(comp_result) == len(accounts_data),
    "Left join behavior - preserves all base records"
)

print("\n📋 Filter Functionality Tests:")

# Test 8: Single tuple equality filter
try:
    active_test = active_accounts_fv.plan().to_pandas()
    active_spark_test = active_accounts_fv.plan().to_spark(spark)
    active_polars_test = active_accounts_fv.plan().to_polars()
    active_statuses = active_test['status'].dropna()
    active_filter_works = (
        all(active_statuses == 'ACTIVE') if len(active_statuses) > 0 else True and
        active_spark_test.count() > 0 and
        len(active_spark_test.columns) > 0 and
        active_polars_test.shape[0] > 0 and
        len(active_polars_test.columns) > 0
    )
    validate_test(active_filter_works, "Single tuple equality filter with Spark/Pandas/Polars output (('status', '==', 'ACTIVE'))")
except Exception as e:
    validate_test(False, f"Single tuple equality filter - Error: {e}")

# Test 9: Range filter using tuple format
try:
    mature_test = mature_users_fv.plan().to_pandas()
    mature_spark_test = mature_users_fv.plan().to_spark(spark)
    mature_polars_test = mature_users_fv.plan().to_polars()
    mature_ages = mature_test['age'].dropna()
    range_filter_works = (
        (all(mature_ages > 30) if len(mature_ages) > 0 else True) and
        mature_spark_test.count() > 0 and
        len(mature_spark_test.columns) > 0 and
        mature_polars_test.shape[0] > 0 and
        len(mature_polars_test.columns) > 0
    )
    validate_test(range_filter_works, "Tuple format range filters with Spark/Pandas/Polars output (('age', '>', 30))")
except Exception as e:
    validate_test(False, f"Tuple range filter - Error: {e}")

# Test 10: IN filter using tuple format
try:
    us_uk_test = us_uk_fv.plan().to_pandas()
    us_uk_spark_test = us_uk_fv.plan().to_spark(spark)
    us_uk_polars_test = us_uk_fv.plan().to_polars()
    countries_test = us_uk_test['country'].dropna().unique()
    in_filter_works = (
        (set(countries_test).issubset({'US', 'UK'}) if len(countries_test) > 0 else True) and
        us_uk_spark_test.count() > 0 and
        len(us_uk_spark_test.columns) > 0 and
        us_uk_polars_test.shape[0] > 0 and
        len(us_uk_polars_test.columns) > 0
    )
    validate_test(in_filter_works, "Tuple format IN filters with Spark/Pandas/Polars output (('country', 'in', ['US', 'UK']))")
except Exception as e:
    validate_test(False, f"Tuple IN filter - Error: {e}")

# Test 11: Multiple tuple filters
try:
    multiple_tuple_test = low_risk_high_credit_fv.plan().to_pandas()
    multiple_spark_test = low_risk_high_credit_fv.plan().to_spark(spark)
    multiple_polars_test = low_risk_high_credit_fv.plan().to_polars()
    credit_scores = multiple_tuple_test['credit_score'].dropna()
    risk_cats = multiple_tuple_test['risk_category'].dropna()
    multiple_works = (
        len(multiple_tuple_test) >= 0 and 
        (all(credit_scores > 700) if len(credit_scores) > 0 else True) and
        (all(risk_cats == 'LOW') if len(risk_cats) > 0 else True) and
        multiple_spark_test.count() >= 0 and
        len(multiple_spark_test.columns) > 0 and
        multiple_polars_test.shape[0] >= 0 and
        len(multiple_polars_test.columns) > 0
    )
    validate_test(multiple_works, "Multiple tuple filters with Spark/Pandas/Polars output [('credit_score', '>', 700), ('risk_category', '==', 'LOW')]")
except Exception as e:
    validate_test(False, f"Multiple tuple filters - Error: {e}")

# Test 12: Complex business scenario with tuple filters
try:
    complex_test = premium_high_spenders_fv.plan().to_pandas()
    complex_spark_test = premium_high_spenders_fv.plan().to_spark(spark)
    complex_polars_test = premium_high_spenders_fv.plan().to_polars()
    account_types = complex_test['account_type'].dropna()
    spending = complex_test['total_spend_90d'].dropna()
    complex_works = (
        len(complex_test) >= 0 and
        (all(account_types == 'PREMIUM') if len(account_types) > 0 else True) and
        (all(spending > 1000) if len(spending) > 0 else True) and
        complex_spark_test.count() >= 0 and
        len(complex_spark_test.columns) > 0 and
        complex_polars_test.shape[0] >= 0 and
        len(complex_polars_test.columns) > 0
    )
    validate_test(complex_works, "Complex tuple format business scenarios with Spark/Pandas/Polars output")
except Exception as e:
    validate_test(False, f"Complex tuple scenarios - Error: {e}")

# Test 13: Complete tuple format showcase
try:
    tuple_showcase_test = tuple_showcase_fv.plan().to_pandas()
    tuple_spark_test = tuple_showcase_fv.plan().to_spark(spark)
    tuple_polars_test = tuple_showcase_fv.plan().to_polars()
    showcase_works = (
        len(tuple_showcase_test) >= 0 and
        tuple_spark_test.count() >= 0 and
        len(tuple_spark_test.columns) > 0 and
        tuple_polars_test.shape[0] >= 0 and
        len(tuple_polars_test.columns) > 0
    )
    validate_test(showcase_works, "Complete tuple format showcase with all operators and Spark/Pandas/Polars output")
except Exception as e:
    validate_test(False, f"Tuple format showcase - Error: {e}")

# Test 14: Polars lazy evaluation validation
try:
    # Test that Polars is using lazy evaluation properly
    polars_plan = comprehensive_fv.plan()
    polars_result = polars_plan.to_polars()
    polars_lazy_works = (
        isinstance(polars_result, pl.DataFrame) and
        polars_result.shape[0] > 0 and
        len(polars_result.columns) > 0
    )
    validate_test(polars_lazy_works, "Polars lazy evaluation implementation working correctly")
except Exception as e:
    validate_test(False, f"Polars lazy evaluation - Error: {e}")

print(f"\n🎯 Test Results: {tests_passed}/{total_tests} passed")

if tests_passed == total_tests:
    print("\n🎉 ALL TESTS PASSED! Feature Store SDK is fully functional! 🎉")
    print("\n✨ SDK Features Validated:")
    print("   ✅ Delta Lake storage format")
    print("   ✅ Automatic multi-table joins")
    print("   ✅ Precise feature selection via projections")
    print("   ✅ Custom join key mapping")
    print("   ✅ Multiple output formats (Spark, Pandas, Polars)")
    print("   ✅ Left/Inner join support")
    print("   ✅ Query plan execution")
    print("   ✅ Feature group management")
    print("   ✅ Feature view creation")
    print("   ✅ Tuple filter format: ('status', '==', 'ACTIVE')  # Clean and concise!")
    print("   ✅ Multiple filters: [('age', '>', 30), ('country', 'in', ['US', 'UK'])]")
    print("   ✅ All operators (==, !=, >, >=, <, <=, in, not_in, is_null, is_not_null)")
    print("   ✅ Complex business scenarios with readable filters")
    print("   ✅ Spark DataFrame output works with all filter types")
    print("   ✅ Pandas DataFrame output works with all filter types")
    print("   ✅ Polars DataFrame output works with all filter types (using lazy evaluation)")
    print("   ✅ Simple, clean API")
else:
    print(f"\n⚠️ {total_tests - tests_passed} tests failed. Please review the implementation.")

print(f"\n📊 Final Statistics:")
print(f"   Feature Groups: 4")
print(f"   Feature Views: {5 + 6}")  # Core views (5) + Filter views (6) 
print(f"   Total Features Available: {sum([len(accounts_data.columns), len(users_data.columns), len(transactions_data.columns), len(risk_data.columns)])}")
print(f"   Sample Records: {len(accounts_data)}")
print(f"   Filter Format: Tuple (clean and concise)")
print(f"   Dictionary filter format: REMOVED ✅ (only tuple format supported)")
print(f"   Polars Implementation: Lazy evaluation ✅ (independent from pandas)")

In [ ]:
# Clean up

In [ ]:
# Clean up Spark session
spark.stop()
print("🧹 Spark session stopped")
print("\n🎊 Feature Store SDK Demo Complete! 🎊")